<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
# ! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 
! pip install -q deepmatch

--2022-09-27 15:09:27--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  27.5MB/s    in 0.2s    

2022-09-27 15:09:27 (27.5 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-27 15:09:27--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [2]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

# Data 

In [3]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [4]:
data.user_id.nunique(), data.movie_id.nunique(), data.head(2)

(46, 2175,    user_id  movie_id  rating  timestamp  \
 0        1      1193       5  978300760   
 1        1       661       3  978302109   
 
                                     title     genres gender  age  occupation  \
 0  One Flew Over the Cuckoo's Nest (1975)      Drama      F    1          10   
 1        James and the Giant Peach (1996)  Animation      F    1          10   
 
      zip  
 0  48067  
 1  48067  )

# Feature Engineering

In [5]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
negsample = 1


In [6]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
feature_max_idx 

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [7]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18
2,1,483,3,978301968,My Fair Lady (1964),12,1,1,9,18
3,1,1881,4,978300275,Erin Brockovich (2000),8,1,1,9,18
4,1,1306,5,978824291,"Bug's Life, A (1998)",3,1,1,9,18


In [8]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')

In [9]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [10]:
train_set, test_set = gen_data_set(data, SEQ_LEN, negsample)
len(train_set), len(test_set)

100%|██████████| 46/46 [00:00<00:00, 145.59it/s]

7 8


(19816, 46)

In [11]:
data.loc[2]

user_id                         1
movie_id                      483
rating                          3
timestamp               978301968
title         My Fair Lady (1964)
genres                         12
gender                          1
age                             1
occupation                      9
zip                            18
Name: 2, dtype: object

In [12]:
print(train_set[0])

# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - constant
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len,  - 50
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating

(12, 987, 0, [2095, 1039, 1852, 260, 2047, 781, 265, 1421, 21, 430, 801, 782, 1197, 869, 1663, 1482, 550, 872, 616, 1921, 645, 492, 663, 686, 2108, 411, 463, 996, 1382, 1352, 104, 1469, 103, 2086, 877, 117, 878, 1664, 783, 231, 754, 1161, 185, 1123, 566, 112, 2169, 976, 860, 46], 50, [15, 11, 15, 5, 1, 1, 1, 1, 8, 5, 6, 1, 4, 1, 1, 1, 2, 8, 1, 8, 1, 8, 1, 1, 1, 1, 15, 1, 11, 5, 1, 1, 1, 8, 1, 11, 1, 1, 1, 1, 1, 1, 1, 15, 1, 15, 15, 2, 1, 15], 6)


In [13]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((19816,), (19816,), (46,), (46,))

In [14]:
train_model_input

{'user_id': array([12, 34, 40, ..., 19, 43, 35]),
 'movie_id': array([ 987,  837,  330, ...,  872, 1756,   96]),
 'hist_movie_id': array([[2095, 1039, 1852, ...,  976,  860,   46],
        [1904, 1649, 2047, ...,  715, 1619,  704],
        [  27,  354,   25, ...,    0,    0,    0],
        ...,
        [1179, 1197,  492, ..., 1196, 1597, 2062],
        [ 209, 1701, 1617, ...,  720,  663,  735],
        [1014, 1443,  365, ...,    0,    0,    0]], dtype=int32),
 'hist_genres': array([[15, 11, 15, ...,  2,  1, 15],
        [ 1,  1,  1, ...,  5,  5,  2],
        [ 8,  1,  2, ...,  0,  0,  0],
        ...,
        [ 8,  4,  8, ...,  3,  5,  8],
        [ 1,  5,  1, ...,  5,  1,  1],
        [ 1,  6,  6, ...,  0,  0,  0]], dtype=int32),
 'hist_len': array([50, 50, 13, ..., 50, 50, 33]),
 'genres': array([6, 8, 5, ..., 8, 1, 7]),
 'gender': array([2, 2, 1, ..., 2, 2, 2]),
 'age': array([3, 4, 3, ..., 3, 3, 2]),
 'occupation': array([ 7,  8, 17, ...,  3,  6,  5]),
 'zip': array([13, 29, 33, ..

# Aggregate Feature and create Embeddings

In [15]:
embedding_dim = 32

In [16]:
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

In [17]:
item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]


In [18]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)


In [19]:
len(sampler_config.item_count)

2176

In [20]:
item_feature_columns[0].vocabulary_size

2176

# Model Training

In [21]:
import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

In [22]:
from deepctr.feature_column import build_input_features
from deepctr.layers import DNN
from deepctr.layers.utils import NoMask, combined_dnn_input
from tensorflow.python.keras.models import Model

from deepmatch.inputs import input_from_feature_columns, create_embedding_matrix
from deepmatch.layers.core import SampledSoftmaxLayer, EmbeddingIndex, PoolingLayer
from deepmatch.utils import get_item_embedding, l2_normalize

user_dnn_hidden_units=(128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
sampler_config=sampler_config
seed=1024

In [35]:
user_features

OrderedDict([('user_id', <tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>),
             ('gender', <tf.Tensor 'gender:0' shape=(None, 1) dtype=int32>),
             ('age', <tf.Tensor 'age:0' shape=(None, 1) dtype=int32>),
             ('occupation',
              <tf.Tensor 'occupation:0' shape=(None, 1) dtype=int32>),
             ('zip', <tf.Tensor 'zip:0' shape=(None, 1) dtype=int32>),
             ('hist_movie_id',
              <tf.Tensor 'hist_movie_id:0' shape=(None, 50) dtype=int32>),
             ('hist_len',
              <tf.Tensor 'hist_len_1:0' shape=(None, 1) dtype=int32>),
             ('hist_genres',
              <tf.Tensor 'hist_genres:0' shape=(None, 50) dtype=int32>)])

In [47]:
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_genres:0' shape=(None, 50) dtype=int32>]

In [55]:
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 144) dtype=float32>

In [65]:
if len(item_feature_columns) > 1:
  raise ValueError("Now YoutubeNN only support 1 item feature like item_id")
item_feature_name = item_feature_columns[0].name
item_vocabulary_size = item_feature_columns[0].vocabulary_size

embedding_matrix_dict = create_embedding_matrix(user_feature_columns + item_feature_columns, l2_reg_embedding,
                                                seed=seed)

user_features = build_input_features(user_feature_columns)
user_inputs_list = list(user_features.values())
user_sparse_embedding_list, user_dense_value_list = input_from_feature_columns(user_features, user_feature_columns,
                                                                                l2_reg_embedding, seed=seed,
                                                                                embedding_matrix_dict=embedding_matrix_dict)
user_dnn_input = combined_dnn_input(user_sparse_embedding_list, user_dense_value_list)

item_features = build_input_features(item_feature_columns)
item_inputs_list = list(item_features.values())

user_dnn_out = DNN(user_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                    dnn_use_bn, output_activation=output_activation, seed=seed)(user_dnn_input)
user_dnn_out = l2_normalize(user_dnn_out)

item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

item_embedding_matrix = embedding_matrix_dict[
    item_feature_name]
item_embedding_weight = NoMask()(item_embedding_matrix(item_index))

pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])

pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
# output = SampledSoftmaxLayer(sampler_config._asdict(), temperature)(
#     [pooling_item_embedding_weight, user_dnn_out, item_features[item_feature_name]])
# model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

# model.__setattr__("user_input", user_inputs_list)
# model.__setattr__("user_embedding", user_dnn_out)

# model.__setattr__("item_input", item_inputs_list)
# model.__setattr__("item_embedding",
#                   get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))

In [67]:
embedding_matrix_dict

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1dfe950>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1e5cb50>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1dfed90>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1dfee10>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1e19210>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1f6a950>,
 'genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f95f1e9ff10>}

In [57]:
item_index

<tf.Tensor 'embedding_index/Const:0' shape=(2176,) dtype=int32>

In [59]:
item_embedding_matrix

In [60]:
item_embedding_weight

<tf.Tensor 'no_mask_3_1/no_mask_3/Identity:0' shape=(2176, 32) dtype=float32>

In [66]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_6/l2_normalize:0' shape=(2176, 32) dtype=float32>

In [ ]:
model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [ ]:
model.user_input_list

In [ ]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )


In [ ]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

In [ ]:
user_embs